# * Adhoc : Target DTAC Sales

## Parameter

In [1]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

## ETL Process

### Step 1 : Import Data Source (Excel)
    Read Excel data

In [11]:
# src_file = 'C:/Ruz/Pentaho/Jobs/Input/Target/ADHOC_RAW_DTAC_SALE_TARGET_AREA.xlsx'
src_file = 'C:\Ruz\Pentaho\Jobs\Input\Target\ADHOC_RAW_DTAC_SALE_TARGET_AREA.xlsx'

# cols = ['USE_FLAG', 'TM_KEY_MTH', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'METRIC_VALUE', 'COMP_CD', 'ORGID_P', 'ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT', 'ORGID_HH', 'D_CLUSTER', 'PROVINCE_ENG', 'REMARK']

src_df = pd.read_excel(src_file, sheet_name='DTAC_Sales_Target', skiprows=2, usecols='N:AD', index_col=None) 
src_df = src_df[src_df['USE_FLAG']=='Y']
src_df = src_df.replace(np.nan, None)
src_df.head(3)

,USE_FLAG,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,METRIC_VALUE,COMP_CD,ORGID_P,ZONE_TYPE,ORGID_G,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,PROVINCE_ENG,REMARK
0,Y,202401,None,DB1R000900,Prepaid Inflow M1 : DTAC,4194780.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),050,"BKK : Bang Khun Thian, Chom Thong, Bang Bon",None,None,None,DTAC : Sales target Jan-May 2024(Ref : K.Vorap...
1,Y,202401,None,DB1R000900,Prepaid Inflow M1 : DTAC,397642.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),016,"BKK : Bangkok Yai, Bangkok Noi, Bang Phlat",None,None,None,DTAC : Sales target Jan-May 2024(Ref : K.Vorap...
2,Y,202401,None,DB1R000900,Prepaid Inflow M1 : DTAC,3269671.0,DTAC,P,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),040,"BKK : Taling Chan, Phasi Charoen, Thawi Wattha...",None,None,None,DTAC : Sales target Jan-May 2024(Ref : K.Vorap...


In [12]:
src_df[['METRIC_CD', 'METRIC_NAME']].drop_duplicates().sort_values('METRIC_CD').reset_index(drop=True)

# Check null value in each column
# src_df.apply(lambda x: sum(x.isnull()), axis=0)

# src_df.columns.ravel()
# src_df['METRIC_NAME'].tolist()

,METRIC_CD,METRIC_NAME
0,DB1R000900,Prepaid Inflow M1 : DTAC
1,DB1R000900AA,Prepaid Inflow M1 : DTAC : Account Executive
2,DB1R000900AB,Prepaid Inflow M1 : DTAC : B2B
3,DB1R000900AC,Prepaid Inflow M1 : DTAC : Branded Retail
4,DB1R000900AD,Prepaid Inflow M1 : DTAC : Contact Center
5,DB1R000900AE,Prepaid Inflow M1 : DTAC : Direct Sales
6,DB1R000900AF,Prepaid Inflow M1 : DTAC : Key Account
7,DB1R000900AG,Prepaid Inflow M1 : DTAC : Modern Trade
8,DB1R000900AH,Prepaid Inflow M1 : DTAC : Others
9,DB1R000900AI,Prepaid Inflow M1 : DTAC : Own Digital


In [6]:
# src_df.dtypes
# src_df.describe()

### Step 2 : ADHOC_RAW_DTAC_SALE_TARGET_AREA
    Truncate -> Insert

In [18]:
# Excel to DB(rawdata)

print(f'\nJob Start...')

# AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected')
cur = conn.cursor()
rows = [tuple(x) for x in src_df.values]


try:
    print(f'\n Processing...')

    # Truncate
    cur.execute("TRUNCATE TABLE AUTOKPI.ADHOC_RAW_DTAC_SALE_TARGET_AREA")
    print(f'\n -> TRUNCATE : "ADHOC_RAW_DTAC_SALE_TARGET_AREA" : Done !')

    # Insert
    cur.executemany("INSERT INTO ADHOC_RAW_DTAC_SALE_TARGET_AREA (USE_FLAG, TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME, METRIC_VALUE, COMP_CD, ORGID_P, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_H, HOP_HINT, ORGID_HH, D_CLUSTER, PROVINCE_ENG, REMARK)\
                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17)", rows)
    print(f'\n -> INSERT : "ADHOC_RAW_DTAC_SALE_TARGET_AREA" : Done !\n\n{src_df.shape[0]} rows, {src_df.shape[1]} columns')
    conn.commit()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    print(f'\nJob Done !!!')


Job Start...

AKPIPRD : Connected

 Processing...

 -> TRUNCATE : "ADHOC_RAW_DTAC_SALE_TARGET_AREA" : Done !

 -> INSERT : "ADHOC_RAW_DTAC_SALE_TARGET_AREA" : Done !

10296 rows, 17 columns

AKPIPRD : Disconnected

Job Done !!!


### Step 3 : ADHOC_DTAC_TARGET_AREA_DAILY
    Delete -> Insert

In [22]:
# Rawdata to Aggregate

print(f'\nJob Start...')

# AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected')
cur = conn.cursor()


try:
    print(f'\n Processing...')

    # Delete
    cur.execute("""
        DELETE AUTOKPI.ADHOC_DTAC_TARGET_AREA_DAILY TGT
                
        WHERE EXISTS (
            SELECT 1 FROM AUTOKPI.ADHOC_RAW_DTAC_SALE_TARGET_AREA SRC
            WHERE TGT.TM_KEY_MTH = SRC.TM_KEY_MTH 
            AND TGT.METRIC_CD = SRC.METRIC_CD )
        OR (METRIC_CD = 'TB4R001000' AND TM_KEY_MTH = 202404) --Adjust Target : TVS Inflow M1(Apr24)
        --OR METRIC_CD = 'TB4R001000'
    """)
    print(f'\n -> DELETE : ADHOC_DTAC_TARGET_AREA_DAILY : Done !!!')

    # Insert
    cur.execute("""
        INSERT INTO AUTOKPI.ADHOC_DTAC_TARGET_AREA_DAILY (TM_KEY_YR, TM_KEY_MTH, TRUE_TM_KEY_WK, TM_KEY_DAY, DAYS_IN_MONTH, PERIODFLAG, PRODUCT_GRP, METRIC_CD, METRIC_NAME, METRIC_GRP, COMP_CD, VERSION, METRIC_VALUE, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, FREQUENCY, REMARK)

        WITH W_RAW_DTAC_SALE_TARGET_FIXED AS 
        (
            SELECT TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, METRIC_VALUE, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, REMARK--, LOAD_DATE
            FROM (
                -->> P : Nationwide
                SELECT TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, SUM(METRIC_VALUE) AS METRIC_VALUE
                    , 1 AS AREA_NO, 'P' AS AREA_TYPE, 'P' AS AREA_CD, 'Nationwide' AS AREA_NAME
                    , REMARK--, MAX(LOAD_DATE) AS LOAD_DATE
                FROM AUTOKPI.ADHOC_RAW_DTAC_SALE_TARGET_AREA NOLOCK
                WHERE USE_FLAG = 'Y'
                GROUP BY TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, REMARK
                
                UNION ALL 
                
                -->> G : 8 Region
                SELECT TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, SUM(METRIC_VALUE) AS METRIC_VALUE
                    , 2 AS AREA_NO, 'G' AS AREA_TYPE, ORGID_G AS AREA_CD, TDS_SGMD AS AREA_NAME
                    , REMARK--, MAX(LOAD_DATE) AS LOAD_DATE
                FROM AUTOKPI.ADHOC_RAW_DTAC_SALE_TARGET_AREA NOLOCK
                WHERE USE_FLAG = 'Y' AND ORGID_P IS NOT NULL
                GROUP BY TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, ORGID_G, TDS_SGMD, REMARK
                
                UNION ALL 
                
                -->> H : 65 HOP_HINT
                SELECT TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, SUM(METRIC_VALUE) AS METRIC_VALUE
                    , 3 AS AREA_NO, 'H' AS AREA_TYPE, ORGID_H AS AREA_CD, HOP_HINT AS AREA_NAME
                    , REMARK--, MAX(LOAD_DATE) AS LOAD_DATE
                FROM AUTOKPI.ADHOC_RAW_DTAC_SALE_TARGET_AREA NOLOCK
                WHERE USE_FLAG = 'Y' AND ORGID_P IS NOT NULL
                GROUP BY TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, ORGID_H, HOP_HINT, REMARK
            )
        ) -->> W_RAW_DTAC_SALE_TARGET_FIXED
        -----------------------------------------------------------------------------------------------------------------------

        , W_ADHOC_DTAC_SALE_TARGET_AREA_MONTHLY AS 
        (
            SELECT P.TM_KEY_YR, P.TM_KEY_MTH, NULL AS TRUE_TM_KEY_WK, NULL AS TM_KEY_DAY, P.DAYS_IN_MONTH, NULL AS PERIODFLAG
                , CASE  WHEN T.METRIC_NAME LIKE '%Prepaid%' THEN 'Prepaid' 
                        WHEN T.METRIC_NAME LIKE '%Postpaid%' THEN 'Postpaid' 
                        WHEN T.METRIC_NAME LIKE '%TOL%' THEN 'TOL' 
                        WHEN T.METRIC_NAME LIKE '%TVS%' THEN 'TVS' 
                        END PRODUCT_GRP
                , T.METRIC_CD, T.METRIC_NAME
                , 'Sales' AS METRIC_GRP
                , T.COMP_CD, 'T' AS VERSION
                , T.METRIC_VALUE, T.AREA_NO, T.AREA_TYPE, T.AREA_CD, T.AREA_NAME
                , 'MTH' AS FREQUENCY
                , T.REMARK--, T.LOAD_DATE
                
            FROM W_RAW_DTAC_SALE_TARGET_FIXED T
            
            LEFT JOIN (SELECT DISTINCT TM_KEY_YR, TM_KEY_MTH, DAYS_IN_MONTH FROM AUTOKPI.DIM_PERIOD) P 
                ON T.TM_KEY_MTH = P.TM_KEY_MTH
        ) -->> W_ADHOC_DTAC_SALE_TARGET_AREA_MONTHLY
        -----------------------------------------------------------------------------------------------------------------------
                
        -->> Output
                
        SELECT TM_KEY_YR, TM_KEY_MTH, TRUE_TM_KEY_WK, TM_KEY_DAY, DAYS_IN_MONTH, PERIODFLAG, PRODUCT_GRP, METRIC_CD, METRIC_NAME, METRIC_GRP, COMP_CD, VERSION, METRIC_VALUE, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, FREQUENCY, REMARK
        FROM W_ADHOC_DTAC_SALE_TARGET_AREA_MONTHLY
                
        UNION ALL 

        -->> Adjust Target : TVS Inflow M1(Apr24)
        SELECT TM_KEY_YR, TM_KEY_MTH, NULL AS TRUE_TM_KEY_WK, NULL AS TM_KEY_DAY
            , DAYS_IN_MONTH, NULL AS PERIODFLAG
            , PRODUCT_GRP, METRIC_CD, METRIC_NAME, METRIC_GRP, COMP_CD, VERSION
            , SUM(TARGET_DAY) AS METRIC_VALUE
            , AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
            , 'MTH' AS FREQUENCY, NULL AS REMARK
        FROM AUTOKPI.VW_SALE_TARGET_AREA_DAILY
        WHERE METRIC_CD = 'TB4R001000' 
        AND TM_KEY_MTH = 202404
        GROUP BY TM_KEY_YR, TM_KEY_MTH, DAYS_IN_MONTH, PRODUCT_GRP, METRIC_CD, METRIC_NAME, METRIC_GRP, COMP_CD, VERSION, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
        
        ORDER BY TM_KEY_MTH, METRIC_GRP, PRODUCT_GRP, METRIC_CD, AREA_NO, AREA_TYPE 
    """)

    conn.commit()
    print(f'\n -> INSERT : ADHOC_DTAC_TARGET_AREA_DAILY : Done !!!')
    # print(f'\n"ADHOC_DTAC_TARGET_AREA_DAILY" : Done !!!\n\n{src_df.shape[0]} rows, {src_df.shape[1]} columns')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')


Job Start...

AKPIPRD : Connected

 Processing...

 -> DELETE : ADHOC_DTAC_TARGET_AREA_DAILY : Done !!!

 -> INSERT : ADHOC_DTAC_TARGET_AREA_DAILY : Done !!!

AKPIPRD : Disconnected
